In [25]:
from importnb import Notebook
import requests
import getpass
import json

with Notebook():
    import db_utils

db_utils.init_db()

Docs table initialized successfully


In [26]:
def upload(filename):
    file_size = None
    try:
        with open(filename, "rb") as file:
            file_hash, file_size = db_utils.get_file_hash(file)
            print(f"file hash we made: {file_hash}")
            file_existance = db_utils.is_uploaded(filename, file_hash)
            print(f"file existance: {file_existance}")
            if (file_existance == "duplicate"):
                print("File already exists")
                return
            elif (file_existance == "modified"):
                print("File exists but is modified recently, Replacing old file...")
                db_utils.update_entry(filename, file_size, file_hash)
            else:
                print("File doesnt exist. Uploading...")
                db_utils.add_entry(filename, file_size, file_hash)
            
            files = {"file": (filename, file, "text/plain")}
            response = requests.post("http://192.168.1.100:8000/upload", files=files)

        print()
        print(json.dumps(response.json(), indent=4))
        print(f"{filename} uploaded to cloud.")
        print()
    except FileNotFoundError:
        print("File not found")

In [27]:
def download(filename):
    params = {"filename": filename}
    response = requests.get("http://192.168.1.100:8000/download", params=params)
    download_dir = os.path.expanduser("~/cloudbox_downloads")
    os.makedirs(download_dir, exist_ok=True)
    if response.status_code == 200:
        os.makedirs(f"/{download_dir}", exist_ok=True)
        with open(f"/{download_dir}/{filename}", "wb") as f:
            f.write(response.content)
    print(f"{filename} downloaded from cloud and saved to cloudbox_downloads.")

In [28]:
def auth():
    password = input("Enter password: ")
    username = getpass.getuser()
    try:
        login_data = {
            "username" : username,
            "password" : password,
            "grant_type" : "password"
        }
        login_response = requests.post("http://192.168.1.100:8000/token", data = login_data)
        if login_response.status_code == 200:
            token_path = os.path.expanduser("~/Library/Application Support/myapp/")
            os.makedirs(token_path, exist_ok=True)
            with open(os.path.join(token_path,"token.json"), "w") as f:
                json.dump(login_response.json(), f)
            print("Authentication Successful")
            return True
        else:
            print("Authentication failed. Error: ", login_response.text)
            return False
    except Exception as e:
        print("Something's wrong in auth. Error: ", e)
        return False

In [29]:
def register_user():
    print("Register yourself first\n")
    username = getpass.getuser()
    password = input("Enter password: ")

    try:
        # call /register here
        data = {
            "username" : username,
            "password" : password
        }
        response = requests.post("http://192.168.1.100:8000/register", json=data)
        if response.status_code == 200:
            print("Registered successfully")
        else:
            print("Registration failed")
            return False

        # call /token immediately
        login_data = {
            "username" : username,
            "password" : password,
            "grant_type" : "password"
        }
        login_response = requests.post("http://192.168.1.100:8000/token", data = login_data)
        if login_response.status_code == 200:
            token_path = os.path.expanduser("~/Library/Application Support/myapp/")
            os.makedirs(token_path, exist_ok=True)
            with open(os.path.join(token_path,"token.json"), "w") as f:
                json.dump(login_response.json(), f)
            print("Authentication Successful")
            return True
        else:
            print("Authentication failed. Error: ", login_response.text)
            return False
    except Exception as e:
        print("Somethin's wrong in register. Error: ", e)
        return False


In [30]:
def check_user_exists(server_url="http://192.168.1.100:8000"):
    try:
        response = requests.get(f"{server_url}/user_exists")
        if response.status_code == 200:
            data = response.json()
            return data.get("user_exists", False)
        else:
            print(f"User does not exist")
            return False
    except Exception as e:
        print(f"Error checking if user exists: {e}")
        return False
